In [ ]:
#pip install pyyaml h5py

In [ ]:
#pip install tensorflow==2.7

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Gerekli Kütüphanelerin Yüklenmesi İşlemi**

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing import image

**Veri Ön İşlemesi**

**Eğitim Setinin Ön İşlemesi**

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

training_set = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ColabNotebooks/FaceShapeDataset/training_set',
    target_size = (64, 64), 
    batch_size = 32,
    class_mode = 'categorical'
)


**Doğrulama Setinin Ön İşlemesi**

In [ ]:
validation_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

validation_set = validation_datagen.flow_from_directory(
    '/content/drive/MyDrive/ColabNotebooks/FaceShapeDataset/validation_set',
    target_size = (64, 64), 
    batch_size = 32,
    class_mode = 'categorical'
)

**Test Setinin Ön İşlemesi**

In [ ]:
test_datagen = ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_set = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ColabNotebooks/FaceShapeDataset/testing_set',
    target_size = (64, 64), 
    batch_size = 32,
    class_mode = 'categorical'
)

**CNN Nesnesi Oluşturulması**

In [ ]:
cnn = tf.keras.models.Sequential()

**Convolution Katman Oluşturma İşlemi**

In [ ]:
cnn.add(
    tf.keras.layers.Conv2D(
       filters = 32,
       kernel_size = 3, 
       activation = 'relu', 
       input_shape = [64, 64, 3]
    )
)

**Pooling İşlemi**

In [ ]:
cnn.add(
    tf.keras.layers.MaxPool2D(
       pool_size = 2,
       strides = 2
    )
)

**İkinci Convolutional Katmanı Oluşturulması**

In [ ]:
cnn.add(
    tf.keras.layers.Conv2D(
       filters = 32,
       kernel_size = 3,
       activation = 'relu'
    )
)

cnn.add(
    tf.keras.layers.MaxPool2D(
       pool_size = 2,
       strides = 2
    )
)

**Flattening İşlemi**

In [ ]:
cnn.add(tf.keras.layers.Flatten())

**Full Connection İşlemi**

In [ ]:
cnn.add(
    tf.keras.layers.Dense(
       units = 128,
       activation = 'relu'
    )
)

**Çıkış Katmanı Oluşturma İşlemi**

In [ ]:
cnn.add(
    tf.keras.layers.Dense(
       units = 5,
       activation = 'sigmoid'
    )
)

In [ ]:
cnn.summary()

**Derleme İşlemi**

In [ ]:
from tensorflow.python import metrics

cnn.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy']
)

**Eğitim İşlemi**

In [ ]:
cnn.fit(
    x = training_set,
    validation_data = validation_set,
    batch_size=16, 
    shuffle=True,
    verbose=1,
    epochs=65
)

**Modelin Kaydedilmesi**

In [ ]:
cnn.save('/content/drive/MyDrive/ColabNotebooks/FaceHairModel_v1.h5')

In [ ]:
#loss, accuracy = cnn.evaluate(test_set)
#print('Test accuracy :', accuracy)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
FaceHairModel = keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/FaceHairModel_v1.h5')

In [ ]:
import urllib
from io import BytesIO

header= { 'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) '
          'AppleWebKit/537.11 (KHTML, like Gecko) '
          'Chrome/23.0.1271.64 Safari/537.11',
          'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.8',
          'Connection': 'keep-alive'}

q = 'https://facehair.000webhostapp.com/yuklenenler/images.jpeg-3.jpg'
req = urllib.request.Request(url=q, headers=header)
page = urllib.request.urlopen(req).read()

img = image.load_img(BytesIO(page), target_size = (64, 64))
img = image.img_to_array(img)
img = np.expand_dims(img , axis = 0)

result = FaceHairModel.predict(img)

sekiller = ["Kalp Yüz Şekli", "Uzun Yüz Şekli", "Oval Yüz Şekli", "Yuvarlak Yüz Şekli", "Kare Yüz Şekli"]
max = result[0][0]
index = 0

for i in range(5):
  if max < result[0][i]:
    max = result[0][i]
    index = i

print(sekiller[index])

1/1 [==============================] - 0s 16ms/step
Oval Yüz Şekli
